In [4]:
!pip install python-chess

In [5]:
import numpy as np
import math
import random

import chess
import chess.pgn

import torch
import torch.nn as nn
import torch.nn.functional as F

from dataclasses import dataclass
from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader
import chess.pgn
import os

import io
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler
import time
import warnings

from torch.amp import autocast, GradScaler
from joblib import Parallel, delayed

# **Encoder Decoder**

In [6]:

def encode(board):
    encoded = torch.zeros((22,8,8), dtype = torch.float32)
    piece_map = {
        chess.PAWN: 0, chess.KNIGHT: 1, chess.BISHOP: 2, chess.ROOK: 3, chess.QUEEN: 4, chess.KING: 5
    }
    
    #12 channels -> 6 for white pieces[0-5] + 6 for black pieces[6-11]
    for square, piece in board.piece_map().items():
        offset = 6 if piece.color == chess.BLACK else 0
        piece_channel = piece_map[piece.piece_type] + offset
        rank, file = divmod(square, 8)
        encoded[piece_channel, rank, file] = 1
    
    #4 channels for castling rights[12-15]
    encoded[12,:,:] = board.has_kingside_castling_rights(chess.WHITE)
    encoded[13,:,:] = board.has_queenside_castling_rights(chess.WHITE)
    encoded[14,:,:] = board.has_kingside_castling_rights(chess.BLACK)
    encoded[15,:,:] = board.has_queenside_castling_rights(chess.BLACK)

    #1 channel en passant [16]
    if board.ep_square:
        ep_rank, ep_file = divmod(board.ep_square,8)
        encoded [16,ep_rank,ep_file] = 1

    #1 channel player turn -> 1 for white 0 for black
    encoded[17,:,:] = board.turn

    # Halfmove clock and fullmove number (normalized) -> 2 channels[18,19] 
    encoded[18, :, :] = board.halfmove_clock / 50.0
    encoded[19, :, :] = board.fullmove_number / 100.0

    # Threefold repetition rule -> 2 channels[20,21]
    encoded[20, :, :] = board.is_repetition(2)
    encoded[21, :, :] = board.is_repetition(3)

    return encoded     #-> 12+4+1+1+2+2 = 22 channels


def show_encoding(encoded):
    C,R,F = encoded.shape
    print(C,R,F)
    for i in range(C):
        print('\n'+ '='*40 + '\n')
        print(f'channel no: {i}')
        print(encoded[i,:,:])



def encode_action(piece, initial_pos, final_pos, underpromote = None):
    i, j = initial_pos
    x, y = final_pos
    dx, dy = x - i, y - j
    idx = None
    
    if piece in ['P', 'R', 'B', 'Q', 'K', 'p', 'r', 'b', 'q', 'k'] and underpromote is None:
        if dx != 0 and dy == 0: #idx=[0,1,2,3,4,5,6]black  idx=[7,8,9,10,11,12,13]white   North-South
            idx = 7 + dx if dx < 0 else 6 + dx
        elif dx == 0 and dy != 0:  #idx=[14,15,16,17,18,19,20]West->East idx=[21,22,23,24,25,26,27]East->West  
            idx = 21 + dy if dy < 0 else 20 + dy
        elif dx == dy:  # idx[28,29,30,31,32,33,34]SW->NE   idx[35,36,37,38,39,40,41]SW<-NE
            idx = 35 + dx if dx < 0 else 34 + dx
        elif dx == -dy:  # idx[42,43,44,45,46,47,48]SE->NW   idx[49,50,51,52,53,54,55]SE<-NW
            idx = 49 + dx if dx < 0 else 48 + dx


    elif piece in ["n", "N"]:  # Knight moves idx=[56,57,58,59,60,61,62,63]
        knight_moves = {(2, -1): 56, (2, 1): 57, (1, -2): 58, (-1, -2): 59,
                        (-2, 1): 60, (-2, -1): 61, (-1, 2): 62, (1, 2): 63}
        idx = knight_moves.get((dx, dy))
    
    elif piece in ["p", "P"] and (x == 0 or x == 7) and underpromote is not None:  # Underpromotions idx = [64, 65, 66, 67, 68, 69, 70, 71, 72, 73,74,75]
        # print('this is triggered')
        underpromotion_map = {
            (4, 0): 64,   # ROOK,  straight ahead
            (2, 0): 65,   # KNIGHT, straight ahead
            (3, 0): 66,   # BISHOP, straight ahead
            (5, 0): 67,   # QUEEN,  straight ahead
            (4, -1): 68,  # ROOK,   capture left
            (2, -1): 69,  # KNIGHT, capture left
            (3, -1): 70,  # BISHOP, capture left
            (5, -1): 71,  # QUEEN,  capture left
            (4, 1): 72,   # ROOK,   capture right
            (2, 1): 73,   # KNIGHT, capture right
            (3, 1): 74,   # BISHOP, capture right
            (5, 1): 75    # QUEEN,  capture right
        }
        
        idx = underpromotion_map.get((underpromote, dy))

    if idx is not None: 
        return idx, i, j
    else:
        return ValueError(f"Some error in encode function ")


def decode_action(encoded):
    encoded = encoded.view(76,8,8)
    C,R,F = torch.where(encoded > 0)
    C, R, F = C.tolist(), R.tolist(), F.tolist()
    probabilities = encoded[C, R, F].tolist()
    prom, i_pos, f_pos = [],[],[]

    for c,r,f in zip(C,R,F):
        initial_pos = (r,f)
        final_pos = None
        promoted = None

        if 0 <= c <= 13:
            dx = c - 7 if c < 7 else c - 6
            final_pos = (r + dx, f)
        elif 14 <= c <= 27:
            dy = c - 21 if c < 21 else c - 20
            final_pos = (r, f + dy)
        elif 28 <= c <= 41:
            dy = c - 35 if c < 35 else c - 34
            final_pos = (r + dy, f + dy)
        elif 42 <= c <= 55:
            dx = c - 49 if c < 49 else c - 48
            dy = -dx
            final_pos = (r + dx, f + dy)
        elif 56 <= c <= 63:  
            knight_moves = {
                56: (r+2, f-1), 57: (r+2, f+1), 58: (r+1, f-2), 59: (r-1, f-2),
                60: (r-2, f+1), 61: (r-2, f-1), 62: (r-1, f+2), 63: (r+1, f+2)
            }
            final_pos = knight_moves[c]
        elif 64<=c<=75:
            underpromote_map = {
                64: ('rook', 0), 65: ('knight', 0), 66: ('bishop', 0), 67: ('queen', 0),
                68: ('rook', -1), 69: ('knight', -1), 70: ('bishop', -1), 71: ('queen', -1),
                72: ('rook', 1), 73: ('knight', 1), 74: ('bishop', 1), 75: ('queen', 1),
            }
            promoted, dy = underpromote_map[c]
            final_pos = (r,f+dy)
        
        i_pos.append(initial_pos)
        f_pos.append(final_pos)
        prom.append(promoted)

    return i_pos, f_pos, prom, probabilities



def encode_legal_moves(board, log):
    legal_mask = torch.zeros((76, 8, 8), dtype=torch.uint8)
    for move in board.legal_moves:
        i, j = divmod(move.from_square, 8)
        x, y = divmod(move.to_square, 8)
        piece = board.piece_at(move.from_square)
        
        if piece is not None:  
            idx, x, y = encode_action(piece.symbol(), (i, j), (x, y), move.promotion)
            if log: print(f"Encoding move: {piece} {move} -> idx {idx}, position ({x}, {y})")            
            legal_mask[idx, x, y] = 1
            if log: print(legal_mask[idx,:,:])
    return legal_mask.view(76*8*8)


def decode_from_index(index: int) -> chess.Move:
    c, board = divmod(index, 64)
    r,f = divmod(board,8)
    initial_pos = (r,f)
    final_pos = None
    promoted = None

    if 0 <= c <= 13:
        dx = c - 7 if c < 7 else c - 6
        final_pos = (r + dx, f)
    elif 14 <= c <= 27:
        dy = c - 21 if c < 21 else c - 20
        final_pos = (r, f + dy)
    elif 28 <= c <= 41:
        dy = c - 35 if c < 35 else c - 34
        final_pos = (r + dy, f + dy)
    elif 42 <= c <= 55:
        dx = c - 49 if c < 49 else c - 48
        dy = -dx
        final_pos = (r + dx, f + dy)
    elif 56 <= c <= 63:  
        knight_moves = {
            56: (r+2, f-1), 57: (r+2, f+1), 58: (r+1, f-2), 59: (r-1, f-2),
            60: (r-2, f+1), 61: (r-2, f-1), 62: (r-1, f+2), 63: (r+1, f+2)
        }
        final_pos = knight_moves[c]
    elif 64<=c<=75:
        underpromote_map = {
            64: (4, 0),   # ROOK,  straight ahead
            65: (2, 0),   # KNIGHT, straight ahead
            66: (3, 0),   # BISHOP, straight ahead
            67: (5, 0),   # QUEEN,  straight ahead
            68: (4, -1),  # ROOK,   capture left
            69: (2, -1),  # KNIGHT, capture left
            70: (3, -1),  # BISHOP, capture left
            71: (5, -1),  # QUEEN,  capture left
            72: (4, 1),   # ROOK,   capture right
            73: (2, 1),   # KNIGHT, capture right
            74: (3, 1),   # BISHOP, capture right
            75: (5, 1),   # QUEEN,  capture right
        }
        dx = -1 if initial_pos[0]==1 else 1
        promoted, dy = underpromote_map[c]
        final_pos = (r+dx,f+dy)

    # print(f'initial_position: {initial_pos}')
    # print(f'final_position: {final_pos}')
    # print(f'promotes: {promoted}')

    return make_move(initial_pos, final_pos, promoted)



def make_move(initial_pos, final_pos, promoted=None):
    from_square = chess.square(initial_pos[1], initial_pos[0])
    to_square = chess.square(final_pos[1], final_pos[0])
    promotion_piece = promoted
    return chess.Move(from_square, to_square, promotion=promotion_piece)



In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        assert self.config.n_channels is not None
        assert self.config.n_filters is not None

        self.input_conv = nn.Conv2d(config.n_channels, config.n_filters, kernel_size=3, padding=1)
        self.input_bn = nn.BatchNorm2d(config.n_filters)
        
        self.residual_tower = nn.ModuleList([Block(config) for _ in range(config.n_BLOCKS)])
        
        self.policy_head = PolicyHead(config)
        
        self.value_head = ValueHead(config)
        
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        nn.init.normal_(self.value_head.fc2.weight, 0, 0.01)
        nn.init.constant_(self.value_head.fc2.bias, 0)
        nn.init.normal_(self.policy_head.fc.weight, 0, 0.01)
        nn.init.constant_(self.policy_head.fc.bias, 0)

    def forward(self, input, targets=None):
        x = F.relu(self.input_bn(self.input_conv(input)))

        for block in self.residual_tower:
            x = block(x)

        value = self.value_head(x)
        policy = self.policy_head(x)
        
        if targets is not None:
            policy_target = targets['policy']
            policy = policy.masked_fill(targets['legal_mask'] == 0, -6e4)
            policy_loss = F.cross_entropy(policy, policy_target)
            value_target = targets['value']
            value_loss = F.mse_loss(value, value_target)
            loss = value_loss + policy_loss
        else:
            loss = None
            policy_loss = None
            value_loss = None

        return value, policy, loss, policy_loss, value_loss


class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.conv1 = nn.Conv2d(config.n_filters, config.n_filters, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(config.n_filters)
        self.conv2 = nn.Conv2d(config.n_filters, config.n_filters, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(config.n_filters)
        self.se = SELayer(config.n_filters, config.SE_channels)

    def forward(self, x):
        residual = x
        
        x = F.relu(self.bn1(x))
        x = self.conv1(x)
        x = F.relu(self.bn2(x))
        x = self.conv2(x)
        
        x = self.se(x)
        
        return residual + x


class SELayer(nn.Module):
    def __init__(self, n_filters, se_channels):
        super().__init__()
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(n_filters, se_channels)
        self.fc2 = nn.Linear(se_channels, 2 * n_filters)
        self.relu = nn.ReLU()

    def forward(self, x):
        B, C, _, _ = x.size()
        pooled = self.global_pool(x).view(B, C)
        out = self.relu(self.fc1(pooled))
        out = self.fc2(out)
        scale, bias = torch.split(out, C, dim=1)
        scale = torch.sigmoid(scale).view(B, C, 1, 1)
        bias = bias.view(B, C, 1, 1)
        return x * scale + bias



class PolicyHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.conv = nn.Conv2d(config.n_filters, config.n_filters, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(config.n_filters)
        self.conv2 = nn.Conv2d(config.n_filters, config.policy_channels, kernel_size=3, padding=1)
        self.fc = nn.Linear(config.policy_channels * 8 * 8, config.policy_channels * 8 * 8)

    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


class ValueHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.conv = nn.Conv2d(config.n_filters, 32, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return torch.tanh(self.fc2(x)).squeeze(1)


@dataclass
class ModelConfig:
    n_channels: int = 22
    n_filters: int = 128
    n_BLOCKS: int = 10  
    SE_channels: int = 64  
    policy_channels: int = 76

def test_neural_network():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    config = ModelConfig(
        n_channels=22,
        n_filters=128,
        n_BLOCKS=10,         
        SE_channels=64,
        policy_channels=76
    )
    
    model = NeuralNetwork(config).to(device)
    
    dummy_input = torch.randn(4, 22, 8, 8).to(device)
    k = torch.zeros(4,76*8*8)
    k[:][3]=1
    
    value, policy, *_ = model(dummy_input)
    assert value.shape == (4,), f"Value shape {value.shape} != (4,)"
    assert policy.shape == (4, 76*8*8), f"Policy shape {policy.shape} != (4, {76*8*8})"
    
    dummy_targets = {
        'value': torch.randn(4).to(device),              
        'policy': torch.randint(0, 76*8*8, (4,)).to(device)  ,
        'legal_mask': k.to(device)
    }
    value, policy, loss, policy_loss, value_loss = model(dummy_input, dummy_targets)
    assert not torch.isnan(loss), "Loss is NaN"
    
    print("All tests passed!")
    print("Value output shape:", value.shape)          
    print("Policy output shape:", policy.shape)        
    print("Loss:", loss.item())
    print("Policy loss:", policy_loss.item())
    print("Value loss:", value_loss.item())

test_neural_network()

All tests passed!
Value output shape: torch.Size([4])
Policy output shape: torch.Size([4, 4864])
Loss: 8.905374526977539
Policy loss: 8.325251579284668
Value loss: 0.5801228284835815


# **Loading model**

In [8]:
scaler = GradScaler()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = ModelConfig()
model = NeuralNetwork(config).to(device)
model = torch.nn.DataParallel(model)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

def load_model(checkpoint_path):
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("...Loaded model and optimizer from checkpoint")
    else:
        print("Training from scratch...", '\n')

checkpoint_path = '/kaggle/input/chessnet10/pytorch/default/1/chess_model (9).pt'
load_model(checkpoint_path)

Training from scratch... 



# **Dataloader**

In [9]:
class ChessDataset(Dataset):
    def __init__(self, buffer):
        
        self.buffer = buffer

    def __len__(self):
        return len(self.buffer)

    def __getitem__(self, idx):
        board_tensor, (idx_action, x, y), value, legal_mask = self.buffer[idx]
        val = idx_action * 64 + 8 * x + y
        return (
            board_tensor.float(), 
            torch.tensor(val, dtype=torch.long),
            torch.tensor([value], dtype=torch.float32),
            legal_mask.float()
        )

In [ ]:
# data_buffer = []
# game_count = 0
# pgn_path = '/kaggle/input/ccrldataset/CCRL.pgn'
# pgn = open(pgn_path)
# load_model(checkpoint_path)

# while True:
#     start = time.time()
#     game = chess.pgn.read_game(pgn)
#     if game is None:
#         break
#     if(game_count<0):
#         game = chess.pgn.read_game(pgn_path)
#         game_count += 1
#     else:
#         game_count += 1
#         result = {'1-0': 1, '0-1': -1, '1/2-1/2': 0}[game.headers['Result']]
#         board = game.board()
    
#         for move in game.mainline_moves():
#             encoded_board = encode(board).to(device)
#             i, j = divmod(move.from_square, 8)
#             x, y = divmod(move.to_square, 8)
#             piece = board.piece_at(move.from_square)
#             idx_action, x, y = encode_action(piece.symbol(), (i, j), (x, y), move.promotion)
#             legal_mask = encode_legal_moves(board, log=False)
#             data_buffer.append((encoded_board, (idx_action, x, y), result, legal_mask))
#             board.push(move)
    
        
    
#         if game_count % 1000 == 0:
#             print(f'\n[INFO] Training after {game_count} games ({len(data_buffer)} positions)')
#             dataset = ChessDataset(data_buffer)
#             loader = DataLoader(dataset, batch_size=64, shuffle=True)
        
#             model.train()
#             running_loss = 0
#             p_running_loss = 0
#             v_running_loss = 0
#             pbar = tqdm(loader, desc=f"Epoch @ {game_count} games, time:{(time.time()-start):.2f}", dynamic_ncols=True)
        
#             for i, (boards, policy_targets, values, legal_mask) in enumerate(pbar):
#                 boards = boards.to(device)
#                 policy_targets = policy_targets.to(device)
#                 values = values.squeeze(-1).to(device)
        
#                 targets = {
#                     'value': values,
#                     'policy': policy_targets,
#                     'legal_mask': legal_mask
#                 }
        
#                 optimizer.zero_grad()
#                 with autocast(device_type='cuda'):
#                     _, _, loss, p_loss, v_loss = model(boards, targets)
#                     loss = loss.mean()
        
#                 scaler.scale(loss).backward()
#                 scaler.step(optimizer)
#                 scaler.update()
        
#                 running_loss += loss.item()
#                 p_running_loss += p_loss.mean().item()
#                 v_running_loss += v_loss.mean().item()
        
#                 avg_loss = running_loss / (i + 1)
#                 p_avg_loss = p_running_loss / (i + 1)
#                 v_avg_loss = v_running_loss / (i + 1)
        
#                 pbar.set_postfix(
#                     total_loss=f'{avg_loss:.4f}',
#                     policy_loss=f'{p_avg_loss:.2f}',
#                     value_loss=f'{v_avg_loss:.2f}'
#                 )
        
#             torch.save({'model_state_dict': model.state_dict(),
#                         'optimizer_state_dict': optimizer.state_dict()}, 
#                        '/kaggle/working/chess_model.pt')
#             data_buffer.clear()


In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        assert self.config.n_channels is not None
        assert self.config.n_filters is not None

        self.input_conv = nn.Conv2d(config.n_channels, config.n_filters, kernel_size=3, padding=1)
        self.input_bn = nn.BatchNorm2d(config.n_filters)
        
        self.residual_tower = nn.ModuleList([Block(config) for _ in range(config.n_BLOCKS)])
        
        self.policy_head = PolicyHead(config)
        
        self.value_head = ValueHead(config)
        
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        nn.init.normal_(self.value_head.fc2.weight, 0, 0.01)
        nn.init.constant_(self.value_head.fc2.bias, 0)
        nn.init.normal_(self.policy_head.fc.weight, 0, 0.01)
        nn.init.constant_(self.policy_head.fc.bias, 0)

    def forward(self, input, targets=None):
        x = F.relu(self.input_bn(self.input_conv(input)))

        for block in self.residual_tower:
            x = block(x)

        # value = self.value_head(x)
        value = torch.zeros(1, device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        policy = self.policy_head(x)
        
        if targets is not None:
            policy_target = targets['policy']
            policy = policy.masked_fill(targets['legal_mask'] == 0, -6e4)
            policy_loss = F.cross_entropy(policy, policy_target)
            # value_target = targets['value']
            # value_loss = F.mse_loss(value, value_target)
            value_loss = None
            loss = policy_loss
        else:
            loss = None
            policy_loss = None
            value_loss = None

        return  value, policy, loss, policy_loss, value_loss


class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.conv1 = nn.Conv2d(config.n_filters, config.n_filters, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(config.n_filters)
        self.conv2 = nn.Conv2d(config.n_filters, config.n_filters, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(config.n_filters)
        self.se = SELayer(config.n_filters, config.SE_channels)

    def forward(self, x):
        residual = x
        
        x = F.relu(self.bn1(x))
        x = self.conv1(x)
        x = F.relu(self.bn2(x))
        x = self.conv2(x)
        
        x = self.se(x)
        
        return residual + x


class SELayer(nn.Module):
    def __init__(self, n_filters, se_channels):
        super().__init__()
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(n_filters, se_channels)
        self.fc2 = nn.Linear(se_channels, 2 * n_filters)
        self.relu = nn.ReLU()

    def forward(self, x):
        B, C, _, _ = x.size()
        pooled = self.global_pool(x).view(B, C)
        out = self.relu(self.fc1(pooled))
        out = self.fc2(out)
        scale, bias = torch.split(out, C, dim=1)
        scale = torch.sigmoid(scale).view(B, C, 1, 1)
        bias = bias.view(B, C, 1, 1)
        return x * scale + bias



class PolicyHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.conv = nn.Conv2d(config.n_filters, config.n_filters, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(config.n_filters)
        self.conv2 = nn.Conv2d(config.n_filters, config.policy_channels, kernel_size=3, padding=1)
        self.fc = nn.Linear(config.policy_channels * 8 * 8, config.policy_channels * 8 * 8)

    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


class ValueHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.conv = nn.Conv2d(config.n_filters, 32, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return torch.tanh(self.fc2(x)).squeeze(1)


@dataclass
class ModelConfig:
    n_channels: int = 22
    n_filters: int = 128
    n_BLOCKS: int = 10  
    SE_channels: int = 64  
    policy_channels: int = 76

# def test_neural_network():
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
#     config = ModelConfig(
#         n_channels=22,
#         n_filters=128,
#         n_BLOCKS=10,         
#         SE_channels=64,
#         policy_channels=76
#     )
    
#     model = NeuralNetwork(config).to(device)
    
#     dummy_input = torch.randn(4, 22, 8, 8).to(device)
#     k = torch.zeros(4,76*8*8)
#     k[:][3]=1
    
#     value, policy, *_ = model(dummy_input)
#     assert value.shape == (4,), f"Value shape {value.shape} != (4,)"
#     assert policy.shape == (4, 76*8*8), f"Policy shape {policy.shape} != (4, {76*8*8})"
    
#     dummy_targets = {
#         'value': torch.randn(4).to(device),              
#         'policy': torch.randint(0, 76*8*8, (4,)).to(device)  ,
#         'legal_mask': k.to(device)
#     }
#     value, policy, loss, policy_loss, value_loss = model(dummy_input, dummy_targets)
#     assert not torch.isnan(loss), "Loss is NaN"
    
#     print("All tests passed!")
#     print("Value output shape:", value.shape)          
#     print("Policy output shape:", policy.shape)        
#     print("Loss:", loss.item())
#     print("Policy loss:", policy_loss.item())
#     print("Value loss:", value_loss.item())

# test_neural_network()

In [15]:

scaler = GradScaler()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


config = ModelConfig()
model = NeuralNetwork(config).to(device)

if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

def load_model(checkpoint_path, game_count=0):
    if os.path.exists(checkpoint_path):
        print("Loaded from checkpoint")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    else:
        print("Training from scratch")



checkpoint_path = '/kaggle/working/chess_model.pt'
load_model(checkpoint_path)


class ChessDataset(Dataset):
    def __init__(self, buffer):
        self.buffer = buffer

    def __len__(self):
        return len(self.buffer)

    def __getitem__(self, idx):
        board_tensor, (idx_action, x, y), value, legal_mask = self.buffer[idx]
        val = idx_action * 64 + 8 * x + y
        return (
            board_tensor.float(), torch.tensor(val, dtype=torch.long),torch.tensor([value], dtype=torch.float32),legal_mask.float())

def process_game_lines(pgn_str):
    game = chess.pgn.read_game(io.StringIO(pgn_str))
    if game is None or game.headers.get("Result") not in ["1-0", "0-1", "1/2-1/2"]:
        return []

    result = {"1-0": 1, "0-1": -1, "1/2-1/2": 0}[game.headers["Result"]]
    board = game.board()
    samples = []
    
    for move in game.mainline_moves():
        encoded_board = encode(board)
        i, j = divmod(move.from_square, 8)
        x, y = divmod(move.to_square, 8)
        piece = board.piece_at(move.from_square)
        idx_action, x, y = encode_action(piece.symbol(), (i, j), (x, y), move.promotion)
        legal_mask = encode_legal_moves(board, log=False)
        samples.append((encoded_board, (idx_action, x, y), result, legal_mask))
        board.push(move)

    return samples

def stream_games_by_marker(path, batch_size=1000, skip_games=0):
    with open(path, 'r') as f:
        buffer = []
        current_game = []
        games_processed = 0
        total_games = 0
        
        if skip_games > 0:
            print(f"Skipping first {skip_games} games...")
            for line in f:
                if line.startswith('[Event "CCRL 40/15"]'):
                    if games_processed >= skip_games:
                        current_game.append(line)
                        total_games = games_processed + 1
                        break
                    games_processed += 1
                    current_game = []
                else:
                    if games_processed < skip_games:
                        continue
        
        for line in f:
            if line.startswith('[Event "CCRL 40/15"]'):
                if current_game:
                    buffer.append("".join(current_game))
                    total_games += 1
                    current_game = []
                    if len(buffer) == batch_size:
                        yield buffer, total_games
                        buffer = []
            current_game.append(line)
        
        if current_game:
            buffer.append("".join(current_game))
            total_games += 1
        if buffer:
            yield buffer, total_games


pgn_path = '/kaggle/input/ccrl-dataset/CCRL.pgn'
game_count = 0
batch_size = 1000
star_from_game = 242000


for game_batch, batch_last_game_num in stream_games_by_marker(pgn_path, batch_size=batch_size, skip_games=250000):
    print(f"\nProcessing games {batch_last_game_num - len(game_batch) + 1} to {batch_last_game_num}")
    results = Parallel(n_jobs=4, backend='loky')(delayed(process_game_lines)(g) for g in game_batch)
    data_buffer = [item for sublist in results for item in sublist]
    
    if not data_buffer:
        continue

    dataset = ChessDataset(data_buffer)
    loader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)


    model.train()
    running_loss = 0
    p_running_loss = 0
    # v_running_loss = 0
    
    pbar = tqdm(loader, desc=f"Training on {len(data_buffer)} positions", dynamic_ncols=True)
    for i, (boards, policy_targets, values, legal_mask) in enumerate(pbar):
        
        boards = boards.to(device)
        policy_targets = policy_targets.to(device)
        values = values.squeeze(-1).to(device)
        legal_mask = legal_mask.to(device)
        
        targets = {
            'value': values,
            'policy': policy_targets,
            'legal_mask': legal_mask
        }

        optimizer.zero_grad()
        with autocast(device_type='cuda'):
            _, _, loss, p_loss, _ = model(boards, targets)
            loss = loss.mean()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        p_running_loss += p_loss.mean().item()
        # v_running_loss += v_loss.mean().item()

        pbar.set_postfix({
            'total': f"{running_loss / (i + 1):.3f}",
            'policy': f"{p_running_loss / (i + 1):.2f}",
            # 'value': f"{v_running_loss / (i + 1):.3f}",
            'games': game_count
        })

    game_count += 1000

    torch.save({'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()}, '/kaggle/working/chess_model.pt')
    print(f"\nSaved checkpoint at game {game_count}")


    del data_buffer, dataset, loader, game_batch, results
    torch.cuda.empty_cache()

Loaded from checkpoint


/tmp/ipykernel_31/4028674425.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Skipping first 250000 games...

Processing games 250002 to 251001


Training on 134513 positions: 100%|██████████| 2102/2102 [02:50<00:00, 12.35it/s, total=1.879, policy=1.88, games=0]



Saved checkpoint at game 1000

Processing games 251002 to 252001


...

Training on 148751 positions: 100%|██████████| 2325/2325 [03:14<00:00, 11.93it/s, total=1.857, policy=1.86, games=99000]



Saved checkpoint at game 100000

Processing games 350002 to 351001


Training on 141650 positions: 100%|██████████| 2214/2214 [02:59<00:00, 12.34it/s, total=1.846, policy=1.85, games=1e+5]



Saved checkpoint at game 101000

Processing games 351002 to 352001


KeyboardInterrupt: 

In [ ]:
def stream_games_by_marker(path, batch_size=1000, skip_games=0):
    with open(path, 'r') as f:
        buffer = []
        current_game = []
        games_processed = 0
        total_games = 0
        
        if skip_games > 0:
            print(f"Skipping first {skip_games} games...")
            for line in f:
                if line.startswith('[Event "CCRL 40/15"]'):
                    if games_processed >= skip_games:
                        current_game.append(line)
                        total_games = games_processed + 1
                        break
                    games_processed += 1
                    current_game = []
                else:
                    if games_processed < skip_games:
                        continue
        
        for line in f:
            if line.startswith('[Event "CCRL 40/15"]'):
                if current_game:
                    buffer.append("".join(current_game))
                    total_games += 1
                    current_game = []
                    if len(buffer) == batch_size:
                        yield buffer, total_games - 1
                        buffer = []
            current_game.append(line)
        
        if current_game:
            buffer.append("".join(current_game))
            total_games += 1
        if buffer:
            yield buffer, total_games - 1

pgn_path = '/kaggle/input/ccrl-dataset/CCRL.pgn'

for i, (game_batch, n) in enumerate(stream_games_by_marker(pgn_path, batch_size=10, skip_games=9)):
    print(f"\n=== Batch {i+1} , {n} ===")
    for j, game_str in enumerate(game_batch):
        print(f"\n--- Game {j+1} in Batch {i+1} ---\n")
        # print(game_str[:500])
    if i >= 1:
        break

In [16]:
%%time
import os
import io
import torch
import chess.pgn
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from joblib import Parallel, delayed
from torch.amp import autocast, GradScaler

scaler = GradScaler()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


config = ModelConfig()
model = NeuralNetwork(config).to(device)
model = torch.nn.DataParallel(model)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

class ChessDataset(Dataset):
    def __init__(self, buffer):
        self.buffer = buffer

    def __len__(self):
        return len(self.buffer)

    def __getitem__(self, idx):
        board_tensor, (idx_action, x, y), value, legal_mask = self.buffer[idx]
        val = idx_action * 64 + 8 * x + y
        return board_tensor.float(), torch.tensor(val), torch.tensor(value, dtype=torch.float32), legal_mask


def process_game_lines(pgn_str):
    game = chess.pgn.read_game(io.StringIO(pgn_str))
    
    if game is None or game.headers.get("Result") not in ["1-0", "0-1", "1/2-1/2"]:
        return []

    result = {"1-0": 1, "0-1": -1, "1/2-1/2": 0}[game.headers["Result"]]
    board = game.board()
    samples = []
    move_no = 0
    for move in game.mainline_moves():
        # print(move_no)
        # move_no+=1
        encoded_board = encode(board)
        i, j = divmod(move.from_square, 8)
        x, y = divmod(move.to_square, 8)
        piece = board.piece_at(move.from_square)
        idx_action, x, y = encode_action(piece.symbol(), (i, j), (x, y), move.promotion)
        legal_mask = encode_legal_moves(board, log=False)
        samples.append((encoded_board, (idx_action, x, y), result, legal_mask))
        board.push(move)

    return samples


def stream_games_by_marker(path, batch_size=2):
    with open(path, 'r') as f:
        buffer = []
        current_game = []
        for line in f:
            if line.startswith('[Event "CCRL 40/15"]'):
                if current_game:
                    buffer.append("".join(current_game))
                    current_game = []
                    if len(buffer) == batch_size:
                        yield buffer
                        buffer = []

            current_game.append(line)

        if current_game:
            buffer.append("".join(current_game))
        if buffer:
            yield buffer


pgn_path = '/kaggle/input/ccrl-dataset/CCRL.pgn'
game_count = 0

for game_batch in stream_games_by_marker(pgn_path, batch_size=100):
    start = time.time()
    results = Parallel(n_jobs=4, backend='loky')(
        delayed(process_game_lines)(g) for g in game_batch
    )
    data_buffer = [item for sublist in results for item in sublist]
    game_count += len(game_batch)
    
    if not data_buffer:
        continue

    print(f'\n[INFO] Training after {game_count} games ({len(data_buffer)} positions)')
    dataset = ChessDataset(data_buffer)
    loader = DataLoader(dataset, batch_size=64, shuffle=True)
    print(f"Created dataset with {len(loader) * 64} samples")
    break
    data_buffer.clear()



[INFO] Training after 100 games (13953 positions)
Created dataset with 14016 samples
CPU times: user 5.49 s, sys: 176 ms, total: 5.67 s
Wall time: 7.41 s


In [ ]:
import chess
import chess.pgn
import torch


from datetime import datetime
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch
import chess
import math
import numpy as np
import copy
from collections import defaultdict
from dataclasses import dataclass
import pickle
import datetime

from typing import List, Tuple



class Node():
    def __init__(self, board: chess.Board, parent=None, move=None, device='cpu'):
        self.device = device
        self.board = board
        self.move = move
        self.parent = parent
        self.children = {}
        self.child_priors = torch.zeros(4864, dtype=torch.float32, device=device)
        self.n = torch.zeros(4864, dtype=torch.float32, device=device)
        self.v = torch.zeros(4864, dtype=torch.float32, device=device)
        self.legal_moves = None
        self.is_expanded = False
        self.action_idxes = []

    @property
    def node_n(self):
        return self.parent.n[self.move]

    @node_n.setter
    def node_n(self, value):
        self.parent.n[self.move] = value

    @property
    def node_v(self):
        return self.parent.v[self.move]

    @node_v.setter
    def node_v(self, value):
        self.parent.v[self.move] = value

    def child_Q(self):
        return self.v / (1 + self.n)

    def child_U(self):
        return math.sqrt(self.node_n) * (3.0 * abs(self.child_priors) / (1 + self.n))

    def best_child(self):
        combined = self.child_Q() + self.child_U()
        if self.action_idxes != []:
            bestmove = self.action_idxes[torch.argmax(combined[self.action_idxes])].item()
        else:
            bestmove = torch.argmax(combined).item()
        return bestmove

    def select(self):
        current = self
        while current.is_expanded:
            best_move = current.best_child()
            current = current.maybe_add_child(best_move)
        return current

    def maybe_add_child(self, move):
        if move not in self.children:
            board_copy = copy.deepcopy(self.board)
            actual_move = decode_from_index(move)
            board_copy.push(actual_move)
            self.children[move] = Node(board_copy, move=move, parent=self, device=self.device)
        return self.children[move]

    def add_dirichlet_noise(self, action_idxs: torch.Tensor, child_priors: torch.Tensor) -> torch.Tensor:
        valid_child_priors = child_priors[action_idxs]
        noise = np.random.dirichlet(alpha=np.full(valid_child_priors.shape[0], 0.3))
        noise_tensor = torch.from_numpy(noise).float().to(valid_child_priors.device)
        valid_child_priors = 0.75 * valid_child_priors + 0.25 * noise_tensor
        child_priors[action_idxs] = valid_child_priors
        return child_priors

    def expand(self, child_priors):
        self.is_expanded = True
        self.legal_moves = list(self.board.legal_moves)

        if not self.legal_moves:
            self.is_expanded = False
            return

        legal_move_mask = encode_legal_moves(self.board, False).to(self.device)
        action_idxs = torch.where(legal_move_mask == 1)[0]
        self.action_idxes = action_idxs

        masked_priors = child_priors.to(self.device) * legal_move_mask
        masked_priors = torch.clamp(masked_priors, min=1e-8)
        if self.parent is not None and self.parent.parent is None:
            masked_priors = self.add_dirichlet_noise(action_idxs, masked_priors)
        self.child_priors = masked_priors

    def backup(self, value_estimate: float):
        current = self
        while current.parent is not None:
            current.node_n += 1
            if current.board.turn == chess.WHITE:
                current.node_v += value_estimate
            else:
                current.node_v -= value_estimate
            current = current.parent


class DummyNode:
    def __init__(self, device='cpu'):
        self.parent = None
        self.v = defaultdict(lambda: torch.tensor(0.0, device=device))
        self.n = defaultdict(lambda: torch.tensor(0.0, device=device))


def UCT_Search(game_state, num_reads, net, config, device='cpu'):
    root = Node(game_state, move=None, parent=DummyNode(device), device=device)
    net.eval()
    net.to(device)

    for i in range(num_reads):
        if config.debug: print('==================================')
        if config.debug: print(f'Iteration {i}\n')

        leaf = root.select()
        if config.debug: print(f'Selected leaf with is_expanded={leaf.is_expanded}: \n{leaf.board}')

        encoded_s = encode(leaf.board).unsqueeze(0).to(device)
        v, p, _, _, _ = net(encoded_s)
        p = p.squeeze(0).to(device)
        v = v.item()

        if config.debug: print(f'Policy shape: {p.shape}')
        if config.debug: print(f'Value: {v}')

        if leaf.board.is_game_over():
            leaf.backup(v)
        else:
            leaf.expand(p)
            leaf.backup(v)

    return torch.argmax(root.n), root


def get_policy(root):
    policy = torch.zeros(4864, dtype=torch.float32, device=root.n.device)
    total_visits = root.n.sum()
    if total_visits > 0:
        for idx in torch.where(root.n != 0)[0]:
            policy[idx] = root.n[idx] / total_visits
    return policy






def MCTS_self_play(model, num_games, device, config):
    for i in range(num_games):
        # print(f'<=======game no {i}=============>')
        current_board = chess.Board()
        dataset = []
        value = 0
        i=0
        while not current_board.is_game_over():
            print('move no',i)
            i+=1
            board_state = encode(current_board).to(device)
            best_move, root = UCT_Search(current_board, 100, model, config)
            best_move = best_move.item()
            move = decode_from_index(best_move)
            current_board.push(move)

            policy = get_policy(root)
            dataset.append([board_state.cpu(), policy.cpu()])

        value = {'1-0':1, '0-1':-1, '1/2-1/2': 0}.get(current_board.result(),0)

        dataset_p = []
        for idx,(s,p) in enumerate(dataset):
            if idx == 0:
                dataset_p.append([s,p,0])
            else:
                dataset_p.append([s,p,value])
        del dataset

        filename = f"selfplay_game_{i}_{datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.pkl"
        with open(filename, "wb") as f:
            pickle.dump(dataset_p, f)

        print(f"Saved game {i} to {filename}")




def generate_game(model, device) -> List[Tuple[torch.Tensor, torch.Tensor, float]]:
    board = chess.Board()
    game_data = []
    mcts_config = MCTSConfig()
    i=0
    while not board.is_game_over() and i < mcts_config.MAX_MOVES:
        i+=1
        print(f'move no. {i}')
        board_state = encode(board).to(device)
        best_move, root = UCT_Search(board, mcts_config.NUM_SIMULATIONS, model, mcts_config)
        best_move = best_move.item()
        move = decode_from_index(best_move)
        board.push(move)
        policy = get_policy(root).cpu()
        game_data.append([board_state, policy])
    
    value = {'1-0':1, '0-1':-1, '1/2-1/2': 0}.get(board.result(),0)
    return [(s, p, value) for s, p in game_data]


@dataclass
class ModelConfig:
    n_channels: int = 22
    n_filters: int = 128
    n_BLOCKS: int = 10  
    SE_channels: int = 64  
    policy_channels: int = 76

@dataclass
class MCTSConfig:
    debug = False
    NUM_SIMULATIONS = 500




class SelfPlayPGNGenerator:
    def __init__(self, model):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.config = ModelConfig()
        self.mcts_config = MCTSConfig()
        self.model = model
        self.model.eval()
    
    
    
    def play_game(self, num_simulations=200):
        """Play one full game and return the PGN"""
        board = chess.Board()
        game = chess.pgn.Game()
        game.headers["Event"] = "Self-play"
        game.headers["Date"] = "9"
        game.headers["White"] = "AI"
        game.headers["Black"] = "AI"
        node = game
        i=0
        while not board.is_game_over():
            print(i)
            i+=1
            print(game)
            best_move, _ = UCT_Search(
                board,
                num_simulations,
                self.model,
                self.mcts_config,
                device
            )
            
            move = decode_from_index(best_move.item())
            board.push(move)
            
            node = node.add_variation(move)
        
        game.headers["Result"] = board.result()
        return game
    
    def save_pgn(self, pgn):
        print(pgn)


scaler = GradScaler() 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = ModelConfig()

# Create raw model first
raw_model = NeuralNetwork(config)
raw_model.to(device)

# Wrap it if using multiple GPUs
model = torch.nn.DataParallel(raw_model)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

def load_model(model, optimizer):
    checkpoint_path = '/kaggle/working/chess_model.pt'
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("Loaded model and optimizer from checkpoint")
    else:
        print("Training from scratch")

load_model(model, optimizer)

player = SelfPlayPGNGenerator(model)
game = player.play_game(num_simulations=300)
player.save_pgn(game)
    

In [ ]:
import chess
import chess.pgn
import torch


from datetime import datetime
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch
import chess
import math
import numpy as np
import copy
from collections import defaultdict
from dataclasses import dataclass
import pickle
import datetime

from typing import List, Tuple



class Node():
    def __init__(self, board: chess.Board, parent=None, move=None, device='cpu'):
        self.device = device
        self.board = board
        self.move = move
        self.parent = parent
        self.children = {}
        self.child_priors = torch.zeros(4864, dtype=torch.float32, device=device)
        self.n = torch.zeros(4864, dtype=torch.float32, device=device)
        self.v = torch.zeros(4864, dtype=torch.float32, device=device)
        self.legal_moves = None
        self.is_expanded = False
        self.action_idxes = []

    @property
    def node_n(self):
        return self.parent.n[self.move]

    @node_n.setter
    def node_n(self, value):
        self.parent.n[self.move] = value

    @property
    def node_v(self):
        return self.parent.v[self.move]

    @node_v.setter
    def node_v(self, value):
        self.parent.v[self.move] = value

    def child_Q(self):
        return self.v / (1 + self.n)

    def child_U(self):
        return math.sqrt(self.node_n) * (3.0 * abs(self.child_priors) / (1 + self.n))

    def best_child(self):
        combined = self.child_Q() + self.child_U()
        if self.action_idxes != []:
            bestmove = self.action_idxes[torch.argmax(combined[self.action_idxes])].item()
        else:
            bestmove = torch.argmax(combined).item()
        return bestmove

    def select(self):
        current = self
        while current.is_expanded:
            best_move = current.best_child()
            current = current.maybe_add_child(best_move)
        return current

    def maybe_add_child(self, move):
        if move not in self.children:
            board_copy = copy.deepcopy(self.board)
            actual_move = decode_from_index(move)
            board_copy.push(actual_move)
            self.children[move] = Node(board_copy, move=move, parent=self, device=self.device)
        return self.children[move]

    def add_dirichlet_noise(self, action_idxs: torch.Tensor, child_priors: torch.Tensor) -> torch.Tensor:
        valid_child_priors = child_priors[action_idxs]
        noise = np.random.dirichlet(alpha=np.full(valid_child_priors.shape[0], 0.3))
        noise_tensor = torch.from_numpy(noise).float().to(valid_child_priors.device)
        valid_child_priors = 0.75 * valid_child_priors + 0.25 * noise_tensor
        child_priors[action_idxs] = valid_child_priors
        return child_priors

    def expand(self, child_priors):
        self.is_expanded = True
        self.legal_moves = list(self.board.legal_moves)

        if not self.legal_moves:
            self.is_expanded = False
            return

        legal_move_mask = encode_legal_moves(self.board, False).to(self.device)
        action_idxs = torch.where(legal_move_mask == 1)[0]
        self.action_idxes = action_idxs

        masked_priors = child_priors.to(self.device) * legal_move_mask
        masked_priors = torch.clamp(masked_priors, min=1e-8)
        if self.parent is not None and self.parent.parent is None:
            masked_priors = self.add_dirichlet_noise(action_idxs, masked_priors)
        self.child_priors = masked_priors

    def backup(self, value_estimate: float):
        current = self
        while current.parent is not None:
            current.node_n += 1
            if current.board.turn == chess.WHITE:
                current.node_v += value_estimate
            else:
                current.node_v -= value_estimate
            current = current.parent


class DummyNode:
    def __init__(self, device='cpu'):
        self.parent = None
        self.v = defaultdict(lambda: torch.tensor(0.0, device=device))
        self.n = defaultdict(lambda: torch.tensor(0.0, device=device))


def UCT_Search(game_state, num_reads, net, config, device='cpu'):
    root = Node(game_state, move=None, parent=DummyNode(device), device=device)
    net.eval()

    for i in range(num_reads):
        if config.debug: print('==================================')
        if config.debug: print(f'Iteration {i}\n')

        leaf = root.select()
        if config.debug: print(f'Selected leaf with is_expanded={leaf.is_expanded}: \n{leaf.board}')

        encoded_s = encode(leaf.board).unsqueeze(0).to(device)
        v, p, _, _, _ = net(encoded_s)
        p = p.squeeze(0).to(device)
        v = v.item()

        if config.debug: print(f'Policy shape: {p.shape}')
        if config.debug: print(f'Value: {v}')

        if leaf.board.is_game_over():
            leaf.backup(v)
        else:
            leaf.expand(p)
            leaf.backup(v)

    return torch.argmax(root.n), root


def get_policy(root):
    policy = torch.zeros(4864, dtype=torch.float32, device=root.n.device)
    total_visits = root.n.sum()
    if total_visits > 0:
        for idx in torch.where(root.n != 0)[0]:
            policy[idx] = root.n[idx] / total_visits
    
    return policy









def generate_game(model, device) -> List[Tuple[torch.Tensor, torch.Tensor, float]]:
    board = chess.Board()
    game_data = []
    mcts_config = MCTSConfig()
    game = chess.pgn.Game()
    node = game  # Keep track of the current node in the PGN tree
    i = 0

    while not board.is_game_over():
        i += 1
        print(f"\nMove no. {i}")
        
        board_state = encode(board).to(device)
        best_move, root = UCT_Search(board, mcts_config.NUM_SIMULATIONS, model, mcts_config)
        max_prior_idx = root.action_idxes[torch.argmax(root.child_priors[root.action_idxes])].item()
        move = decode_from_index(best_move.item())
        # Print visit counts, values, and priors only for legal moves
        print("Legal moves and their statistics:\n")
        for idx in root.action_idxes:
            movei = decode_from_index(idx.item())
            visits = root.n[idx].item()
            value = root.v[idx].item()
            prior = root.child_priors[idx].item()
            print(f"Move: {movei}, Index: {idx.item()}, Visits: {visits:.1f}, Value Sum: {value:.2f}, Prior: {prior:.4f}")

        board.push(move)
        node = node.add_variation(move)  # Add the move to the PGN tree

        # Print the current board and PGN after each move
        print(board)
        print("\nPGN so far:")
        print(game)

        policy = get_policy(root).cpu()
        game_data.append([board_state, policy])
        break

    value = {'1-0': 1, '0-1': -1, '1/2-1/2': 0}.get(board.result(), 0)
    return [(s, p, value) for s, p in game_data]



@dataclass
class ModelConfig:
    n_channels: int = 22
    n_filters: int = 128
    n_BLOCKS: int = 10 
    SE_channels: int = 64  
    policy_channels: int = 76

@dataclass
class MCTSConfig:
    debug = False
    NUM_SIMULATIONS = 500



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = ModelConfig()

# Create raw model first
raw_model = NeuralNetwork(config)
raw_model.to(device)

# Wrap it if using multiple GPUs
model = torch.nn.DataParallel(raw_model)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

def load_model(model, optimizer):
    checkpoint_path = '/kaggle/input/chessnet10/pytorch/default/1/chess_model (9).pt'
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("Loaded model and optimizer from checkpoint")
    else:
        print("Training from scratch")

load_model(model, optimizer)




generate_game(model, device)



In [5]:
import torch.nn.functional as F
from dataclasses import dataclass

class NeuralNetwork(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        assert self.config.n_channels is not None
        assert self.config.n_filters is not None

        self.input_conv = nn.Conv2d(config.n_channels, config.n_filters, kernel_size=3, padding=1)
        self.input_bn = nn.BatchNorm2d(config.n_filters)
        
        self.residual_tower = nn.ModuleList([Block(config) for _ in range(config.n_BLOCKS)])
        
        self.policy_head = PolicyHead(config)
        
        self.value_head = ValueHead(config)
        
        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Final layer initialization
        nn.init.normal_(self.value_head.fc2.weight, 0, 0.01)
        nn.init.constant_(self.value_head.fc2.bias, 0)
        nn.init.normal_(self.policy_head.fc.weight, 0, 0.01)
        nn.init.constant_(self.policy_head.fc.bias, 0)

    def forward(self, input, targets=None):
        x = F.relu(self.input_bn(self.input_conv(input)))

        for block in self.residual_tower:
            x = block(x)

        value = self.value_head(x)
        policy = self.policy_head(x)
        
        if targets is not None:
            policy_target = targets['policy']
            policy_loss = F.cross_entropy(policy, policy_target)
            value_loss = F.mse_loss(value, targets['value'])
            loss = value_loss + policy_loss
        else:
            loss = None
            policy_loss = None
            value_loss = None

        return value, policy, loss, policy_loss, value_loss


class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.conv1 = nn.Conv2d(config.n_filters, config.n_filters, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(config.n_filters)
        self.conv2 = nn.Conv2d(config.n_filters, config.n_filters, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(config.n_filters)
        self.se = SELayer(config.n_filters, config.SE_channels)

    def forward(self, x):
        residual = x
        
        # Pre-activation residual block
        x = F.relu(self.bn1(x))
        x = self.conv1(x)
        x = F.relu(self.bn2(x))
        x = self.conv2(x)
        
        # Squeeze-excitation
        x = self.se(x)
        
        return residual + x


class SELayer(nn.Module):
    def __init__(self, n_filters, se_channels):
        super().__init__()
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(n_filters, se_channels)
        self.fc2 = nn.Linear(se_channels, 2 * n_filters)
        self.relu = nn.ReLU()

    def forward(self, x):
        B, C, _, _ = x.size()
        pooled = self.global_pool(x).view(B, C)
        out = self.relu(self.fc1(pooled))
        out = self.fc2(out)
        scale, bias = torch.split(out, C, dim=1)
        scale = torch.sigmoid(scale).view(B, C, 1, 1)
        bias = bias.view(B, C, 1, 1)
        return x * scale + bias



class PolicyHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.conv = nn.Conv2d(config.n_filters, config.n_filters, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(config.n_filters)
        self.conv2 = nn.Conv2d(config.n_filters, config.policy_channels, kernel_size=3, padding=1)
        self.fc = nn.Linear(config.policy_channels * 8 * 8, config.policy_channels * 8 * 8)

    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


class ValueHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.conv = nn.Conv2d(config.n_filters, 32, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return torch.tanh(self.fc2(x)).squeeze(1)


@dataclass
class ModelConfig:
    n_channels: int = 22
    n_filters: int = 256
    n_BLOCKS: int = 20  
    SE_channels: int = 64  
    policy_channels: int = 76

def test_neural_network():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    config = ModelConfig(
        n_channels=22,
        n_filters=256,
        n_BLOCKS=20,         
        SE_channels=64,
        policy_channels=76
    )
    
    model = NeuralNetwork(config).to(device)
    
    dummy_input = torch.randn(4, 22, 8, 8).to(device)
    
    value, policy, *_ = model(dummy_input)
    assert value.shape == (4,), f"Value shape {value.shape} != (4,)"
    assert policy.shape == (4, 76*8*8), f"Policy shape {policy.shape} != (4, {76*8*8})"
    
    dummy_targets = {
        'value': torch.randn(4).to(device),              
        'policy': torch.randint(0, 76*8*8, (4,)).to(device) 
    }
    value, policy, loss, policy_loss, value_loss = model(dummy_input, dummy_targets)
    assert not torch.isnan(loss), "Loss is NaN"
    
    print("All tests passed!")
    print("Value output shape:", value.shape)          
    print("Policy output shape:", policy.shape)        
    print("Loss:", loss.item())
    print("Policy loss:", policy_loss.item())
    print("Value loss:", value_loss.item())

test_neural_network()

All tests passed!
Value output shape: torch.Size([4])
Policy output shape: torch.Size([4, 4864])
Loss: 10.338056564331055
Policy loss: 10.01834487915039
Value loss: 0.3197115659713745


In [ ]:
scaler = GradScaler() 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = ModelConfig()
model = NeuralNetwork(config).to(device)
model = torch.nn.DataParallel(model)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

def load_model():
    checkpoint_path = '/kaggle/input/chessnet20/pytorch/default/1/chess_model (13).pt'
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("Loaded model and optimizer from checkpoint")
    else:
        print("Training from scratch")

load_model()

class ChessDataset(Dataset):
    def __init__(self, buffer):
        self.buffer = buffer  

    def __len__(self):
        return len(self.buffer)

    def __getitem__(self, idx):
        board_tensor, (idx_action, x, y), value = self.buffer[idx]
        val = idx_action * 64 + 8 * x + y
        return board_tensor.float(), torch.tensor(val), torch.tensor(value, dtype=torch.float32),

import io


data_buffer = []
game_count = 0
pgn_path = '/kaggle/input/ccrldataset/CCRL.pgn'
pgn = open(pgn_path)


while True:
    game = chess.pgn.read_game(pgn)
    if game is None:
        break
    # if(game_count<= 62000):
    #     game_count += 1
    #     continue
        
    game_count += 1
    result = {'1-0': 1, '0-1': -1, '1/2-1/2': 0}[game.headers['Result']]
    board = game.board()

    for move in game.mainline_moves():
        encoded_board = encode(board).to(device)
        i, j = divmod(move.from_square, 8)
        x, y = divmod(move.to_square, 8)
        piece = board.piece_at(move.from_square)
        idx_action, x, y = encode_action(piece.symbol(), (i, j), (x, y), move.promotion)
        data_buffer.append((encoded_board, (idx_action, x, y), result))
        board.push(move)

    

    if game_count % 1000 == 0:
        print(f'\n[INFO] Training after {game_count} games ({len(data_buffer)} positions)')
        dataset = ChessDataset(data_buffer)
        loader = DataLoader(dataset, batch_size=64, shuffle=True)
    
        model.train()
        running_loss = 0
        p_running_loss = 0
        v_running_loss = 0
        pbar = tqdm(loader, desc=f"Epoch @ {game_count} games", dynamic_ncols=True)
    
        for i, (boards, policy_targets, values) in enumerate(pbar):
            boards = boards.to(device)
            policy_targets = policy_targets.to(device)
            values = values.squeeze(-1).to(device)
    
            targets = {
                'value': values,
                'policy': policy_targets,
            }

            optimizer.zero_grad()
            with autocast(device_type='cuda'):
                _, _, loss, p_loss, v_loss = model(boards, targets)
                loss = loss.mean()
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        
            running_loss += loss.item()
            p_running_loss += p_loss.mean().item()
            v_running_loss += v_loss.mean().item()
            
            avg_loss = running_loss / (i + 1)
            p_avg_loss = p_running_loss / (i + 1)
            v_avg_loss = v_running_loss / (i + 1)
            
            pbar.set_postfix(
                total_loss=f'{avg_loss:.4f}',
                policy_loss=f'{p_avg_loss:.2f}',
                value_loss=f'{v_avg_loss:.2f}'
            )
    
        torch.save({'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict()}, '/kaggle/working/chess_model.pt')
        data_buffer.clear()

In [ ]:
scaler = GradScaler() 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = ModelConfig()

raw_model = NeuralNetwork(config)
raw_model.to(device)

model = torch.nn.DataParallel(raw_model)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

def load_model(model, optimizer):
    checkpoint_path = '/kaggle/working/chess_model.pt'
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("Loaded model and optimizer from checkpoint")
    else:
        print("Training from scratch")

load_model(model, optimizer)

_, p, _, _, _ = net(encoded_s)




In [23]:
import torch
import chess
import chess.pgn
import os
from torch.cuda.amp import GradScaler

# Setup
scaler = GradScaler()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = ModelConfig()

raw_model = NeuralNetwork(config)
raw_model.to(device)
model = torch.nn.DataParallel(raw_model)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

def load_model(model, optimizer):
    checkpoint_path = '/kaggle/working/chess_model.pt'
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("Loaded model and optimizer from checkpoint")
    else:
        print("Training from scratch")

load_model(model, optimizer)

def select_move(board):
    encoded = encode(board)  
    encoded = torch.tensor(encoded, dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        _, policy, _, _, _ = model(encoded)

    legal_move_mask = encode_legal_moves(board, False).to(device)
    masked_policy = policy * legal_move_mask
    move_index = torch.argmax(masked_policy).item()

    move = decode_from_index(move_index)
    return move

board = chess.Board()
game = chess.pgn.Game()
node = game

exporter = chess.pgn.StringExporter(headers=True, variations=False, comments=False)

print(board)

while not board.is_game_over():
    if board.turn == chess.WHITE:
        move_input = input("Your move (e.g. e4, Nf3, or e2e4): ")
        try:
            try:
                move = board.parse_san(move_input)  # Try SAN
            except:
                move = chess.Move.from_uci(move_input)  # Try UCI

            if move in board.legal_moves:
                board.push(move)
                node = node.add_variation(move)
                print("\nPGN after your move:\n" + game.accept(exporter) + "\n")
            else:
                print("Illegal move. Try again.")
                continue
        except Exception:
            print("Invalid input. Try again.")
            continue
    else:
        print("AI is thinking...")
        ai_move = select_move(board)
        print(f"AI plays: {ai_move}")
        board.push(ai_move)
        node = node.add_variation(ai_move)

        print("\nPGN after AI move:\n" + game.accept(exporter) + "\n")

    print(board)

print("Game over:", board.result())


KeyboardInterrupt: Interrupted by user

In [26]:
import torch
import chess
import chess.pgn
import os
import io
from torch.cuda.amp import GradScaler
@dataclass
class ModelConfig:
    n_channels: int = 22
    n_filters: int = 128
    n_BLOCKS: int = 10 
    SE_channels: int = 64  
    policy_channels: int = 76

@dataclass
class MCTSConfig:
    debug = False
    NUM_SIMULATIONS = 500

mcts_config = MCTSConfig()

# Setup
scaler = GradScaler()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = ModelConfig()

raw_model = NeuralNetwork(config)
raw_model.to(device)
model = torch.nn.DataParallel(raw_model)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

def load_model(model, optimizer):
    checkpoint_path = '/kaggle/working/chess_model.pt'
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("Loaded model and optimizer from checkpoint")
    else:
        print("Training from scratch")

load_model(model, optimizer)

# Create board and PGN game structure
board = chess.Board()
game = chess.pgn.Game()
node = game
exporter = chess.pgn.StringExporter(headers=True, variations=False, comments=False)

print(board)

while not board.is_game_over():
    if board.turn == chess.WHITE:
        move_input = input("Your move (e.g. e4, Nf3, or e2e4): ")
        try:
            try:
                move = board.parse_san(move_input)
            except:
                move = chess.Move.from_uci(move_input)

            if move in board.legal_moves:
                board.push(move)
                node = node.add_variation(move)
                print("\nPGN after your move:\n" + game.accept(exporter) + "\n")
            else:
                print("Illegal move. Try again.")
                continue
        except Exception:
            print("Invalid input. Try again.")
            continue
    else:
        print("AI is thinking...")
        board_state = encode(board).to(device)
        best_move_index, _ = UCT_Search(board, mcts_config.NUM_SIMULATIONS, model, mcts_config)
        best_move = decode_from_index(best_move_index.item())

        print(f"AI plays: {best_move}")
        board.push(best_move)
        node = node.add_variation(best_move)
        print("\nPGN after AI move:\n" + game.accept(exporter) + "\n")

    print(board)

print("Game over:", board.result())


/tmp/ipykernel_31/2502072067.py:23: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_31/2502072067.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loa

Loaded model and optimizer from checkpoint
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


Your move (e.g. e4, Nf3, or e2e4):  e4



PGN after your move:
[Event "?"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "?"]
[Black "?"]
[Result "*"]

1. e4 *

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
AI is thinking...


RuntimeError: module must have its parameters and buffers on device cuda:0 (device_ids[0]) but found one of them on device: cpu

In [ ]:
import chess
import chess.pgn
import torch
from torch.cuda.amp import GradScaler
import os
from dataclasses import dataclass

# === Model and MCTS Configs ===
@dataclass
class ModelConfig:
    n_channels: int = 22
    n_filters: int = 128
    n_BLOCKS: int = 10
    SE_channels: int = 64
    policy_channels: int = 76

@dataclass
class MCTSConfig:
    debug: bool = False
    NUM_SIMULATIONS: int = 500

# === Initialize Model and Device ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = GradScaler()
config = ModelConfig()
mcts_config = MCTSConfig()

raw_model = NeuralNetwork(config)
raw_model.to(device)
model = torch.nn.DataParallel(raw_model)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

def load_model(model, optimizer):
    checkpoint_path = '/kaggle/working/chess_model.pt'
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("Loaded model and optimizer from checkpoint")
    else:
        print("Training from scratch")

load_model(model, optimizer)

# === Human vs AI ===
board = chess.Board()
game = chess.pgn.Game()
game.headers["Event"] = "Human vs AI"
game.headers["White"] = "Human"
game.headers["Black"] = "AI"
node = game
exporter = chess.pgn.StringExporter(headers=True, variations=False, comments=False)

print(board)

while not board.is_game_over():
    if board.turn == chess.WHITE:
        move_input = input("Your move (e.g. e4, Nf3, or e2e4): ")
        try:
            try:
                move = board.parse_san(move_input)
            except:
                move = chess.Move.from_uci(move_input)

            if move in board.legal_moves:
                board.push(move)
                node = node.add_variation(move)
                print("\nPGN after your move:\n" + game.accept(exporter) + "\n")
            else:
                print("Illegal move. Try again.")
                continue
        except Exception:
            print("Invalid input. Try again.")
            continue
    else:
        print("AI is thinking...")
        best_move_index, _ = UCT_Search(board, mcts_config.NUM_SIMULATIONS, model, mcts_config, device)
        move = decode_from_index(best_move_index.item())
        board.push(move)
        node = node.add_variation(move)
        print(f"AI plays: {move}")
        print("\nPGN after AI move:\n" + game.accept(exporter) + "\n")

    print(board)

game.headers["Result"] = board.result()
print("Game over:", board.result())


/tmp/ipykernel_31/1750332985.py:24: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_31/1750332985.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loa

Loaded model and optimizer from checkpoint
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


Your move (e.g. e4, Nf3, or e2e4):  e4



PGN after your move:
[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 *

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
AI is thinking...
AI plays: c7c5

PGN after AI move:
[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 *

r n b q k b n r
p p . p p p p p
. . . . . . . .
. . p . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


Your move (e.g. e4, Nf3, or e2e4):  Nf3


...


PGN after your move:
[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 d6 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 d6 3. Bb5+ *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 d6 3. Bb5+ Bd7 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 d6 3. Bb5+ Bd7 4. Bxd7+ *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]


Your move (e.g. e4, Nf3, or e2e4):  h6



PGN after your move:
[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 d6 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 d6 3. Bb5+ *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 d6 3. Bb5+ Bd7 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 d6 3. Bb5+ Bd7 4. Bxd7+ *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]


Your move (e.g. e4, Nf3, or e2e4):  Kxf2



PGN after your move:
[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 d6 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 d6 3. Bb5+ *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 d6 3. Bb5+ Bd7 *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "Human"]
[Black "AI"]
[Result "*"]

1. e4 c5 2. Nf3 d6 3. Bb5+ Bd7 4. Bxd7+ *

[Event "Human vs AI"]
[Site "?"]
[Date "????.??.??"]
